# Emoji Text Search
find emoji closest in meaning to text

In [13]:
%pip install gensim==3.8.3
%pip install emoji_data
%pip install sklearn

You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Using cached scikit_learn-0.24.2-cp39-cp39-macosx_10_13_x86_64.whl (7.3 MB)
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=58280e74e455374257116c8269afbba644de18a8a7d69bd2e70a3e7273d0f492
  Stored in directory: /Users/sebastiankelly/Library/Caches/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip 

In [1]:
import os
import json
import numpy as np
import pandas as pd
import gensim

In [2]:
APP_FOLDER = os.path.dirname('.')
FIXTURES_FOLDER = os.path.join(APP_FOLDER, "src/fixtures")
GOOGLE_NEWS = os.path.join(FIXTURES_FOLDER, "GoogleNews-vectors-negative300-SLIM.bin.gz")
model = gensim.models.KeyedVectors.load_word2vec_format(GOOGLE_NEWS, binary=True)
model.init_sims(replace=True)

In [58]:
from emoji_data import EmojiSequence
import pandas as pd
import numpy as np

d = {'emoji': [], 'description': []}
for (emoji, emoji_meta) in EmojiSequence:
    d['emoji'].append(emoji)
    d['description'].append(emoji_meta.description)

df = pd.DataFrame(d)
df = df[df['description'] != '']

df['description'] = df['description'].str.split(' skin tone').str[0].str.replace(':', '').str.replace(',', '')
df = df.drop_duplicates(subset=['description'])
print(df)

         emoji                description
0       👨‍❤️‍👨  couple with heart man man
1     👨‍❤️‍💋‍👨               kiss man man
2          👨‍👦             family man boy
3        👨‍👦‍👦         family man boy boy
4          👨‍👧            family man girl
...        ...                        ...
3516        🧝🏻                  elf light
3517        🧝🏼           elf medium-light
3518        🧝🏽                 elf medium
3519        🧝🏾            elf medium-dark
3520        🧝🏿                   elf dark

[2339 rows x 2 columns]


In [15]:
def process_words(text):
    words = text.split(" ")
    word_vectors = [ model.syn0norm[model.vocab[word].index] for word in words if word in model.vocab]
    if len(word_vectors) > 0:
        mean_vector = np.array(word_vectors).mean(axis=0)
        unit_vector = gensim.matutils.unitvec(mean_vector).astype(np.float32).tolist()
    else:
        unit_vector = np.zeros(model.vector_size, ).tolist()
    return unit_vector

In [59]:
df['vector'] = df['description'].apply(process_words)
# class_df = pd.DataFrame(class_keys)
# print(df)
# df.to_csv(EMOJI_DATA)
# EMOJI_DATA = os.path.join(FIXTURES_FOLDER, "emoji_vectors.csv")

/var/folders/z8/0bm67qd10m54kz5_n90f0gvxx01fhr/T/ipykernel_10478/2124056185.py:3: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  word_vectors = [ model.syn0norm[model.vocab[word].index] for word in words if word in model.vocab]


In [65]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd
import random
from sklearn.metrics.pairwise import cosine_similarity

def find_similarity_to_search(search_vector):
    def func(emoji_vector):
        b_emoji = np.array(emoji_vector)
        cos_sim = dot(search_vector, b_emoji) / (norm(search_vector) * norm(b_emoji))
        return cos_sim
    return func

search_text = "dog"
search_vector = np.array(process_words(search_text))
similarity_search = find_similarity_to_search(search_vector)
df['similarity'] = df['vector'].apply(similarity_search)
x = df.nlargest(5, 'similarity')
x[['emoji', 'similarity', 'description']]

/var/folders/z8/0bm67qd10m54kz5_n90f0gvxx01fhr/T/ipykernel_10478/2124056185.py:3: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  word_vectors = [ model.syn0norm[model.vocab[word].index] for word in words if word in model.vocab]


,emoji,similarity,description
1644,🐕,1.000000,dog
1448,🌭,0.775372,hot dog
1631,🐈,0.760946,cat
2313,🦮,0.759934,guide dog
1322,🐕‍🦺,0.722582,service dog
